In [ ]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
import re
from tqdm import tqdm

n_games = 9
interactive_mode = False

# Hyperparameters
player_names = ["Thomas", "Emily", "Benjamin", "Sophia", "Victoria", "Kyle"]
roles = [0, 0, 0, 1, 1, 2]  # Adding one seer

model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf",
    2: "seer"
}

for game in tqdm(range(0, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Get all seers for extra information
    seer_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "seer"]

    # Init all the players
    players: Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        extra_info = []
        if name.lower() in werewolf_names:
            extra_info.append("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n")
        if name.lower() in seer_names:
            extra_info.append("You are the seer. You can see the true identity of one player each night.\n")
        
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=extra_info)
        players[name.lower()].init_player(players=player_names)
        
        # If game is interactive, print the intro of the first player
        if interactive_mode:
            print(f"You are a {players[player_names[0].lower()].role}")
            if player_names[0].lower() in werewolf_names:
                print("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you.")
            if player_names[0].lower() in seer_names:
                print("You are the seer. You can see the true identity of one player each night.")
            print("Narrator: The first night was fruitful for the Werewolf, a villager is dead: Bob. Now it is time to debate who to eliminate today!")

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []

    # Debate stage
    while rounds < 15 and not stop:
        rounds += 1
        next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

        if next_to_speak != "vote":
            # If next to speak is first player and interactive mode, then ask for input
            if next_to_speak == player_names[0].lower() and interactive_mode:
                print("Your turn to speak")
                uncensored = "Thomas: " + input()
                censored = re.sub(r"\[[^\]]*\]", "", uncensored)
            else:
                censored, uncensored = players[next_to_speak].get_player_text()
            
            for name in player_names:
                if name.lower() != next_to_speak:
                    players[name.lower()].add_other_text(censored)
        
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)

            if interactive_mode:
                print(censored)
            else:
                print(uncensored)
        else:
            break

    # Seer stage
    seer_discoveries = []
    for seer_name in seer_names:
        if interactive_mode:
            print(f"{seer_name.capitalize()}, you are the seer. Choose one player to see their role.")
            chosen_player = input("Enter the name of the player you want to see: ").strip().lower()
        else:
            # For non-interactive mode, randomly choose a player
            chosen_player = np.random.choice([name.lower() for name in player_names if name.lower() != seer_name])

        chosen_role = players[chosen_player].role
        players[seer_name].current_context.content += f"\n[You have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.]\n"
        seer_discoveries.append((seer_name, chosen_player, chosen_role))
        if interactive_mode:
            print(f"{seer_name.capitalize()}, you have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.")

    # Announce seer's discovery
    for seer_name, chosen_player, chosen_role in seer_discoveries:
        announcement = f"{seer_name.capitalize()} announces: I have discovered that {chosen_player.capitalize()} is a {chosen_role}."
        for name in player_names:
            players[name.lower()].add_other_text(announcement)
        conversation_history.append(announcement)
        conversation_history_uncensored.append(announcement)
        if interactive_mode:
            print(announcement)

    # Vote stage
    votes = {name.lower(): 0 for name in player_names}

    vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
    conversation_history.append(vote_prompt.content)

    print(vote_prompt.content)

    for name in player_names:
        players[name.lower()].add_other_text(vote_prompt.content)

    for voting_name in player_names:
        if voting_name.lower() in werewolf_names:
            players[voting_name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolfs are (including you): {', '.join(werewolf_names)}. If a wolf die, then you also loose!]\n"

        # If next to speak is first player and interactive mode, then ask for input
        if voting_name.lower() == player_names[0].lower() and interactive_mode:
            print("Your turn to vote")
            uncensored = "Thomas: " + input()
            censored = re.sub(r"\[[^\]]*\]", "", uncensored)
        else:
            censored, uncensored = players[voting_name.lower()].get_player_text()

        for name in player_names:
            if name.lower() != voting_name:
                players[name.lower()].add_other_text(censored)
        
        conversation_history.append(censored)
        conversation_history_uncensored.append(uncensored)

        voted_for = ""
        search = censored.lower()
        for name in player_names:
            if name.lower() in search:
                voted_for = name.lower()
                search = search.split(name.lower())[-1]

        try:
            votes[voted_for] += 1
        except KeyError:
            pass
        if interactive_mode:
            print(censored)
        else:
            print(uncensored)
    print(votes)

    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "seer_names": seer_names,  # Add seer names to the JSON dump
        "player_names": player_names,
        "model": model,
        "interactive": interactive_mode,
    }, open(f"../data/final_project_no_custom{game}.json", "w+"), indent=4)

  0%|          | 0/9 [00:00<?, ?it/s]

Sophia: I suggest we start by sharing any suspicions we might have. [I need to steer the conversation in a way that will help me identify the Werewolf.] Let's not waste time, we need to act decisively to protect our village.
Kyle: [Bob was a villager, now we have to be vigilant.] I have my eye on Benjamin, his demeanor seems off, and his silence is unsettling. What do you think, Emily and Victoria?
Victoria: [Benjamin seems shifty, but I need to play this carefully.] I agree with Kyle, Benjamin's silence is suspicious. Emily, what's your take on this?
Thomas: [Benjamin's silence is indeed suspicious, but I sense a hint of deception in Kyle's accusation. I need to tread carefully and observe their reactions before making any hasty decisions.] I think we should hear Benjamin's defense before rushing to judgment. Let's not overlook any potential clues.
Emily: I think we shouldn't be too quick to judge. Let's hear what Benjamin has to say for himself before making any decisions. [We need t

 11%|█         | 1/9 [00:21<02:51, 21.44s/it]

Kyle: [Emily's defensiveness is suspicious, but we can't ignore Sophia's sudden accusation. Need to analyze the motives carefully.] 

I vote for Emily.
{'thomas': 0, 'emily': 5, 'benjamin': 0, 'sophia': 1, 'victoria': 0, 'kyle': 0}
Sophia: I can't believe Bob is gone already. We need to act fast and find the Werewolf among us. I'm watching everyone closely tonight.
Kyle: I agree, Sophia. We need to act swiftly and decisively. It's crucial we make the right decision today to protect the village. I'll be using my powers as the Seer to uncover the truth tonight. Let's not waste any time. [I need to use my visions wisely and steer the conversation towards revealing the Werewolf.]
Victoria: [Bob's death is a tragedy, but we must focus on finding the werewolf. I need to be cautious and observant of everyone's behavior. I'll need to push for information in the upcoming discussion.] "I agree, we can't afford to hesitate. Let's share our suspicions and observations openly. We must be thorough i

 22%|██▏       | 2/9 [00:49<02:57, 25.33s/it]

...

Kyle: [If Benjamin's vision revealed I'm a peasant, then he must be the Seer, which makes me innocent. I need to defend myself without appearing too defensive.]

Kyle: Benjamin claiming to have seen my innocence as the Seer should be proof enough. I must vote for Sophia as she seems to be sowing doubt and dissent among us.
{'thomas': 0, 'emily': 0, 'benjamin': 1, 'sophia': 2, 'victoria': 0, 'kyle': 3}
Sophia: I noticed that Bob is no longer with us. We need to act swiftly and decisively to uncover the werewolf among us. [Emily and I must stick together and deflect any suspicion onto others to survive.] Let's start by sharing any suspicions or observations we have.
Kyle: [Bob was a villager, so the werewolf is still among us. I need to steer this conversation towards someone suspicious.] I agree with Sophia, we can't afford to waste any time. Does anyone have any thoughts on who they find suspicious?
Victoria: [Bob's death is unfortunate, but I need to focus on finding the werewolf. I m

 67%|██████▋   | 6/9 [02:19<01:08, 22.73s/it]

Kyle: [Emily is a werewolf, and she voted for Benjamin. I need to create a strong reasoning for my vote without revealing too much.] Benjamin's defense is weak, and his accusations seem desperate. My vote goes to **Benjamin**.
{'thomas': 0, 'emily': 1, 'benjamin': 3, 'sophia': 2, 'victoria': 0, 'kyle': 0}
Sophia: [Oh no, poor Bob. We've got a werewolf among us. I need to be strategic and observant.]

Sophia: Alright, folks, let's start by sharing any suspicions or strange behavior you might have noticed yesterday. We need to work together to uncover the werewolf. 

Sophia: I noticed Thomas was quite quiet yesterday. Maybe too quiet. Emily, what do you think? Any thoughts on who might be the werewolf?
Kyle: [Hmm, Sophia seems to be taking charge, could she be trying to divert attention away from herself?] 

Kyle: I agree, Thomas was unusually silent. Emily, do you have any suspicions? 

Emily: I'm not sure yet, I want to hear from everyone before making any accusations. Let's not rush t

 78%|███████▊  | 7/9 [02:51<00:51, 25.95s/it]

Kyle: [Analyzing the situation: Thomas claims to have visions, but could he be using this to manipulate the group? Emily has been cautious and observant throughout the game, but is that a facade? I need to consider who benefits more from deception.] 

I vote for Emily.
{'thomas': 2, 'emily': 4, 'benjamin': 0, 'sophia': 0, 'victoria': 0, 'kyle': 0}
Sophia: I can't believe Bob is gone, we need to find the Werewolf fast. Let's start by sharing any suspicions or strange behavior we noticed. [I need to lead the conversation and push for information to expose the Werewolf.]
Sophia: Emily, did you notice anything unusual last night? We need to work together to find the culprit. [Putting pressure on Emily might reveal something important.]
Sophia: Thomas, what are your thoughts on who the Werewolf might be? We can't afford to make a mistake. [I need to gauge Thomas's reaction and see if he gives anything away.]
Sophia: Kyle, do you have any suspicions about anyone? We need to act fast to prote